In [128]:
from telethon import TelegramClient 
import pandas as pd
import asyncio
from datetime import datetime
import requests
import openpyxl
from openpyxl.utils import get_column_letter
import logging
from config_tg import TELEGRAM_API_ID, TELEGRAM_API_HASH, session_name

api_id = TELEGRAM_API_ID
api_hash = TELEGRAM_API_HASH
client = TelegramClient(session_name, api_id, api_hash)



# Настройка логгера
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def generate_telegram_message_link(dialog_name, message_id): #кривые ссылки, разобраться

    """
    Генерирует прямую ссылку на сообщение в Telegram.

    Параметры:
    chat_id (int): ID чата, может быть отрицательным для супергрупп.
    message_id (int): ID сообщения в чате.

    Возвращает:
    str: URL-адрес сообщения в Telegram.
    """
    #return f"https://t.me/c/{abs(chat_id)}/{message_id}"
    return f"https://t.me/{dialog_name}/{message_id}"

async def tg_read(tg_groups, date_begin=None, date_end=None, last_days=None, keyword=None, limit=None, forwardtochat=None):
    messages_data = []
    dialogs = await client.get_dialogs()

    for tg_group in tg_groups:
        target_dialog = next((d for d in dialogs if d.title == tg_group), None)
        
        if target_dialog:
            logger.info(f"Найден диалог: {target_dialog.title}")
            async for message in client.iter_messages(target_dialog, limit=limit, reverse=False):
                message_date = message.date.date()
                if date_begin and date_end:
                    date_begin_parsed = datetime.strptime(date_begin, '%d/%m/%Y').date()
                    date_end_parsed = datetime.strptime(date_end, '%d/%m/%Y').date()
                    if not (date_begin_parsed <= message_date <= date_end_parsed):
                        continue
                if keyword and keyword.lower() not in message.text.lower():
                    continue

                message_link = generate_telegram_message_link(target_dialog.id, message.id)
                if forwardtochat:
                    await message.forward_to(forwardtochat)  
                messages_data.append({
                    'Group': target_dialog.title,
                    'Date': message.date.strftime("%Y-%m-%d %H:%M:%S"),
                    'Message': message.text,
                    'Link': message_link  # Добавляем ссылку на сообщение
                })
              
            await asyncio.sleep(0.5)
        else:
            logger.warning(f"Диалог '{tg_group}' не найден")

    return pd.DataFrame(messages_data)

def append_to_excel(filename, new_data_df): #обновление файла XLS
    try:
        # Загрузка существующего файла, если он существует
        old_data_df = pd.read_excel(filename, index_col=None)
        logger.info(f"Файл {filename} успешно загружен")
        # Объединение старых данных с новыми
        updated_data_df = pd.concat([old_data_df, new_data_df], ignore_index=True)
    except FileNotFoundError:
        # Если файл не существует, просто используем новые данные и создаем новый файл в этой же директории
        updated_data_df = new_data_df
        logger.info(f"Файл {filename} не существует")
    # Запись обновленных данных в файл
    with pd.ExcelWriter(filename, engine='openpyxl', mode='w') as writer:
        updated_data_df.to_excel(writer, index=False)
        logger.info(f"Данные обновлены")

async def list_chats():
    await client.start()
    # Получаем все диалоги
    async for dialog in client.iter_dialogs():
        print(f'Chat: "{dialog.name}" has ID: {dialog.id}')
   

tg_groups=['Management Jobs — вакансии и резюме', 'Аренда квартир']
filename = r'C:\Users\maksim.evdokimov\Documents\!Dev\Telegram\messages.xlsx'

# Использование асинхронного контекстного менеджера для работы с клиентом
async with client:
    new_data = await tg_read(tg_groups, date_begin='22/04/2024', date_end='25/04/2024', keyword='Project',limit=50)
    append_to_excel(filename, new_data)
    await list_chats()

2024-04-27 15:47:44,160 - INFO - Connecting to 149.154.167.51:443/TcpFull...
2024-04-27 15:47:44,242 - INFO - Connection to 149.154.167.51:443/TcpFull complete!
2024-04-27 15:47:44,933 - INFO - Найден диалог: Management Jobs — вакансии и резюме
2024-04-27 15:47:46,047 - WARNING - Диалог 'Аренда квартир' не найден
2024-04-27 15:47:46,097 - INFO - Файл C:\Users\maksim.evdokimov\Documents\!Dev\Telegram\messages.xlsx успешно загружен
2024-04-27 15:47:46,112 - INFO - Данные обновлены
2024-04-27 15:47:46,444 - INFO - Disconnecting from 149.154.167.51:443/TcpFull...
2024-04-27 15:47:46,445 - INFO - Disconnection from 149.154.167.51:443/TcpFull complete!


Chat: "Аренда квартир БАТУМИ" has ID: -1001567736264
Chat: "Management Jobs — вакансии и резюме" has ID: -1001229149302
Chat: "Мтс Ровер Марий Эл" has ID: 7121612864
Chat: "Мой Номер Телефона" has ID: 382226031
